# Rf 1 0 2_Dataimport
Basic functionality: importing data from ROOT TTrees and THx histograms



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:39 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooDataHist.h"
#include "RooGaussian.h"
#include "TCanvas.h"
#include "RooPlot.h"
#include "TTree.h"
#include "TH1D.h"
#include "TRandom.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

In [3]:
TH1 *makeTH1();
TTree *makeTTree();

input_line_28:2:14: warning: empty parentheses interpreted as a function declaration [-Wvexing-parse]
 TH1 *makeTH1();
             ^~
input_line_28:2:14: note: replace parentheses with an initializer to declare a variable
 TH1 *makeTH1();
             ^~
              = nullptr
input_line_28:3:17: warning: empty parentheses interpreted as a function declaration [-Wvexing-parse]
TTree *makeTTree();
                ^~
input_line_28:3:17: note: replace parentheses with an initializer to declare a variable
TTree *makeTTree();
                ^~
                 = nullptr


 A helper function is created: 

In [4]:
%%cpp -d
TH1 *makeTH1()
{
   // Create ROOT TH1 filled with a Gaussian distribution

   TH1D *hh = new TH1D("hh", "hh", 25, -10, 10);
   for (int i = 0; i < 100; i++) {
      hh->Fill(gRandom->Gaus(0, 3));
   }
   return hh;
}

 A helper function is created: 

In [5]:
%%cpp -d
TTree *makeTTree()
{
   // Create ROOT TTree filled with a Gaussian distribution in x and a uniform distribution in y

   TTree *tree = new TTree("tree", "tree");
   Double_t *px = new Double_t;
   Double_t *py = new Double_t;
   tree->Branch("x", px, "x/D");
   tree->Branch("y", py, "y/D");
   for (int i = 0; i < 100; i++) {
      *px = gRandom->Gaus(0, 3);
      *py = gRandom->Uniform() * 30 - 15;
      tree->Fill();
   }
   return tree;
}

---------------------------------------------------
Importing ROOT histograms
 ===================================================

Import th1 into a roodatahist
 ---------------------------------------------------------

Create a root th1 histogram

In [6]:
TH1 *hh = makeTH1();

Declare observable x

In [7]:
RooRealVar x("x", "x", -10, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create a binned dataset that imports contents of th1 and associates its contents to observable 'x'

In [8]:
RooDataHist dh("dh", "dh", x, Import(*hh));

Plot and fit a roodatahist
 ---------------------------------------------------

Make plot of binned dataset showing poisson error bars (roofit default)

In [9]:
RooPlot *frame = x.frame(Title("Imported TH1 with Poisson error bars"));
dh.plotOn(frame);

Fit a gaussian p.d.f to the data

In [10]:
RooRealVar mean("mean", "mean", 0, -10, 10);
RooRealVar sigma("sigma", "sigma", 3, 0.1, 10);
RooGaussian gauss("gauss", "gauss", x, mean, sigma);
gauss.fitTo(dh);
gauss.plotOn(frame);

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mean         0.00000e+00  2.00000e+00   -1.00000e+01  1.00000e+01
     2 sigma        3.00000e+00  9.90000e-01    1.00000e-01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
 FCN=249.349 FROM MIGRAD    STATUS=INITIATE        8 CALLS           9 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX 

Plot and fit a roodatahist with internal errors
 ---------------------------------------------------------------------------------------------

If histogram has custom error (i.e. its contents is does not originate from a poisson process
 but e.g. is a sum of weighted events) you can data with symmetric 'sum-of-weights' error instead
 (same error bars as shown by ROOT)

In [11]:
RooPlot *frame2 = x.frame(Title("Imported TH1 with internal errors"));
dh.plotOn(frame2, DataError(RooAbsData::SumW2));
gauss.plotOn(frame2);

Please note that error bars shown (poisson or sumw2) are for visualization only, the are not used
 in a maximum likelihood fit

 A (binned) ML fit will ALWAYS assume the Poisson error interpretation of data (the mathematical definition
 of likelihood does not take any external definition of errors). Data with non-unit weights can only be correctly
 fitted with a chi^2 fit (see rf602_chi2fit.C)

-----------------------------------------
Importing ROOT TTrees
 =========================================

Import ttree into a roodataset
 -----------------------------------------------------------

In [12]:
TTree *tree = makeTTree();

Define 2nd observable y

In [13]:
RooRealVar y("y", "y", -10, 10);

Construct unbinned dataset importing tree branches x and y matching between branches and roorealvars
 is done by name of the branch/RRV

 Note that ONLY entries for which x,y have values within their allowed ranges as defined in
 RooRealVar x and y are imported. Since the y values in the import tree are in the range [-15,15]
 and RRV y defines a range [-10,10] this means that the RooDataSet below will have less entries than the TTree
 'tree'

In [14]:
RooDataSet ds("ds", "ds", RooArgSet(x, y), Import(*tree));

[#1] INFO:Eval -- RooTreeDataStore::loadValues(ds) Ignored 36 out of range events


Use ascii import/export for datasets
 ------------------------------------------------------------------------------------

In [15]:
{
   // Write data to output stream
   std::ofstream outstream("/tmp/rf102_testData.txt");
   // Optionally, adjust the stream here (e.g. std::setprecision)
   ds.write(outstream);
   outstream.close();
}

Read data from input stream. the variables of the dataset need to be supplied
 to the RooDataSet::read() function.

In [16]:
std::cout << "\n-----------------------\nReading data from ASCII\n";
RooDataSet *dataReadBack =
   RooDataSet::read("/tmp/rf102_testData.txt",
                    RooArgList(x, y), // variables to be read. If the file has more fields, these are ignored.
                    "D"); // Prints if a RooFit message stream listens for debug messages. Use Q for quiet.

dataReadBack->Print("V");

std::cout << "\nOriginal data, line 20:\n";
ds.get(20)->Print("V");

std::cout << "\nRead-back data, line 20:\n";
dataReadBack->get(20)->Print("V");


-----------------------
Reading data from ASCII
[#1] INFO:DataHandling -- RooDataSet::read: reading file /tmp/rf102_testData.txt
[#0] ERROR:DataHandling -- RooDataSet::read(static): read error at line 65
[#1] INFO:DataHandling -- RooDataSet::read: read 64 events (ignored 0 out of range events)
DataStore dataset (/tmp/rf102_testData.txt)
  Contains 64 entries
  Observables: 
    1)           x = 0.0174204  L(-10 - 10)  "x"
    2)           y = 9.46654  L(-10 - 10)  "y"
    3)  blindState = Normal(idx = 0)
  "Blinding State"

Original data, line 20:
  1) RooRealVar:: x = -0.79919
  2) RooRealVar:: y = 0.0106407

Read-back data, line 20:
  1) RooRealVar::          x = -0.79919
  2) RooRealVar::          y = 0.0106407
  3) RooCategory:: blindState = Normal(idx = 0)



Plot datasets with multiple binning choices
 ------------------------------------------------------------------------------------

Print number of events in dataset

In [17]:
ds.Print();

RooDataSet::ds[x,y] = 64 entries


Print unbinned dataset with default frame binning (100 bins)

In [18]:
RooPlot *frame3 = y.frame(Title("Unbinned data shown in default frame binning"));
ds.plotOn(frame3);

Print unbinned dataset with custom binning choice (20 bins)

In [19]:
RooPlot *frame4 = y.frame(Title("Unbinned data shown with custom binning"));
ds.plotOn(frame4, Binning(20));

RooPlot *frame5 = y.frame(Title("Unbinned data read back from ASCII file"));
ds.plotOn(frame5, Binning(20));
dataReadBack->plotOn(frame5, Binning(20), MarkerColor(kRed), MarkerStyle(5));

Draw all frames on a canvas

In [20]:
TCanvas *c = new TCanvas("rf102_dataimport", "rf102_dataimport", 1000, 800);
c->Divide(3, 2);
c->cd(1);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

c->cd(4);
gPad->SetLeftMargin(0.15);
frame3->GetYaxis()->SetTitleOffset(1.4);
frame3->Draw();
c->cd(5);
gPad->SetLeftMargin(0.15);
frame4->GetYaxis()->SetTitleOffset(1.4);
frame4->Draw();
c->cd(6);
gPad->SetLeftMargin(0.15);
frame4->GetYaxis()->SetTitleOffset(1.4);
frame5->Draw();

Draw all canvases 

In [21]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()